# Explore here

It's recommended to use this notebook for exploration purposes.

For example: 

1. You could import the CSV generated by python into your notebook and explore it.
2. You could connect to your database using `pandas.read_sql` from this notebook and explore it.

EL OBJETIVO DE LA LIBRERIA SQLALCHEMY ES HACER DE TRADUCTOR DE SINTAXIS ENTRE SERVIDORES SQL,SUS COMANDOS Y CODIGO PYHTON.
PRIMERO DE TODO INSTALAMOS LAS DEPENDENCIAS MEDIANTE TERMINAL GIT CON PIP.
pip install sqlalchemy python-dotenv psycopg2-binary.
SI hay errores de incompatibilidad tendremos que utilizar diferentes versiones de python hasta encontrar una que no haya errores de compatibilidad entre las dependencias de cada libreria.
UTILIZARE MI PROPIO SERVIDOR POSTGRESQL INSTALADO EN MI SOBREMESA Y PREVIAMENTE CONECTADO MEDIANTE SSL, PREVIAMENTE CONFIGURADO CON PGADMIN4.
LAS CREDENCIALES SE ENLAZARAN A LA LLAMADA DE UNA VARIABLE EXTERNA A ESTE DOCUMENTO, EL MISMO SERA INVISIBLE PARA LOS USUARIOS DE ESTE WORKSPACE EN REMOTO.
PARA ELLO SE ANOTARA EL NOMBRE DEL DOCUMENTO EN EL .GITIGNORE, EL ARCHIVO EN QUESTION ES UN .ENV QUE SE UTILIZA BASICAMENTE COMO ARCHIVO DE CONFIGURACION.
PARA OPERAR CON ESTE TIPO DE ARCHIVO SE UTILIZA LA LIBRERIA DOTENV COMUNMENTE.

In [1]:
import os
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import create_engine, Column, Integer, String, Date, DECIMAL, ForeignKey
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from pathlib import Path
#Paso 1 cargamos datos de las db y le damos formato a las nuevas tablas almacenadas en postgresql del localhost.
current_dir = Path(os.getcwd())
dotenv_path = current_dir / 'user.env'
load_dotenv(dotenv_path=dotenv_path)
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
DATABASE_URL = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(DATABASE_URL)
Base = declarative_base()
Session = sessionmaker(bind=engine)
session = Session()
class Publisher(Base):
    __tablename__ = 'publishers'
    publisher_id = Column(Integer, primary_key=True)
    name = Column(String(255), nullable=False)
class Author(Base):
    __tablename__ = 'authors'
    author_id = Column(Integer, primary_key=True)
    first_name = Column(String(100), nullable=False)
    middle_name = Column(String(50))
    last_name = Column(String(100))
class Book(Base):
    __tablename__ = 'books'
    book_id = Column(Integer, primary_key=True)
    title = Column(String(255), nullable=False)
    total_pages = Column(Integer)
    rating = Column(DECIMAL(4, 2))
    isbn = Column(String(13))
    published_date = Column(Date)
    publisher_id = Column(Integer, ForeignKey('publishers.publisher_id'))
    publisher = relationship("Publisher")
class BookAuthor(Base):
    __tablename__ = 'book_authors'
    book_id = Column(Integer, ForeignKey('books.book_id'), primary_key=True)
    author_id = Column(Integer, ForeignKey('authors.author_id'), primary_key=True)
    book = relationship("Book")
    author = relationship("Author")
Base.metadata.create_all(engine)
if not session.query(Publisher).first():
    publishers_data = [
        Publisher(publisher_id=1, name='O Reilly Media'),
        Publisher(publisher_id=2, name='A Book Apart'),
        Publisher(publisher_id=3, name='A K PETERS'),
        Publisher(publisher_id=4, name='Academic Press'),
        Publisher(publisher_id=5, name='Addison Wesley'),
        Publisher(publisher_id=6, name='Albert&Sweigart'),
        Publisher(publisher_id=7, name='Alfred A Knopf')
    ]
    session.add_all(publishers_data)
    print("Publishers added to session:", publishers_data)
    session.commit()
    print("Publishers committed")

if not session.query(Author).first():
    authors_data = [
        Author(author_id=1, first_name='Merritt', last_name='Eric'),
        Author(author_id=2, first_name='Linda', last_name='Mui'),
        Author(author_id=3, first_name='Alecos', last_name='Papadatos'),
        Author(author_id=4, first_name='Anthony', last_name='Molinaro'),
        Author(author_id=5, first_name='David', last_name='Cronin'),
        Author(author_id=6, first_name='Richard', last_name='Blum'),
        Author(author_id=7, first_name='Yuval', middle_name='Noah', last_name='Harari'),
        Author(author_id=8, first_name='Paul', last_name='Albitz')
    ]
    session.add_all(authors_data)
    print("Authors added to session:", authors_data)
    session.commit()
    print("Authors committed")

if not session.query(Book).first():
    books_data = [
        Book(book_id=1, title='Lean Software Development: An Agile Toolkit', total_pages=240, rating=4.17, isbn='9780320000000', published_date='2003-05-18', publisher_id=5),
        Book(book_id=2, title='Facing the Intelligence Explosion', total_pages=91, rating=3.87, published_date='2013-02-01', publisher_id=7),
        Book(book_id=3, title='Scala in Action', total_pages=419, rating=3.74, isbn='9781940000000', published_date='2013-04-10', publisher_id=1),
        Book(book_id=4, title='Patterns of Software: Tales from the Software Community', total_pages=256, rating=3.84, isbn='9780200000000', published_date='1996-08-15', publisher_id=1),
        Book(book_id=5, title='Anatomy Of LISP', total_pages=446, rating=4.43, isbn='9780070000000', published_date='1978-01-01', publisher_id=3),
        Book(book_id=6, title='Computing machinery and intelligence', total_pages=24, rating=4.17, published_date='2009-03-22', publisher_id=4),
        Book(book_id=7, title='XML: Visual QuickStart Guide', total_pages=269, rating=3.66, isbn='9780320000000', published_date='2009-01-01', publisher_id=5),
        Book(book_id=8, title='SQL Cookbook', total_pages=595, rating=3.95, isbn='9780600000000', published_date='2005-12-01', publisher_id=7),
        Book(book_id=9, title='The Apollo Guidance Computer: Architecture And Operation (Springer Praxis Books / Space Exploration)', total_pages=439, rating=4.29, isbn='9781440000000', published_date='2010-07-01', publisher_id=6),
        Book(book_id=10, title='Minds and Computers: An Introduction to the Philosophy of Artificial Intelligence', total_pages=222, rating=3.54, isbn='9780750000000', published_date='2007-02-13', publisher_id=7)
    ]
    session.add_all(books_data)
    print("Books added to session:", books_data)
    session.commit()
    print("Books committed")

if not session.query(BookAuthor).first():
    book_authors_data = [
        BookAuthor(book_id=1, author_id=1),
        BookAuthor(book_id=2, author_id=8),
        BookAuthor(book_id=3, author_id=7),
        BookAuthor(book_id=4, author_id=6),
        BookAuthor(book_id=5, author_id=5),
        BookAuthor(book_id=6, author_id=4),
        BookAuthor(book_id=7, author_id=3),
        BookAuthor(book_id=8, author_id=2),
        BookAuthor(book_id=10, author_id=1)
    ]
    session.add_all(book_authors_data)
    print("BookAuthors added to session:", book_authors_data)
    session.commit()
    print("BookAuthors committed")
#Fin del paso 1.

In [2]:
import os
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from pathlib import Path
#Paso 2, hacemos los query de cada tabla y los unificamos con JOIN usando sus tablas en comun.
current_dir = Path(os.getcwd())
dotenv_path = current_dir / 'user.env'
load_dotenv(dotenv_path=dotenv_path)
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
DATABASE_URL = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()
create_table_if_not_exists_query = """
    CREATE TABLE IF NOT EXISTS unified_books (
        book_id VARCHAR(255),
        title VARCHAR(255),
        total_pages VARCHAR(255),
        rating VARCHAR(255),
        isbn VARCHAR(13),
        published_date VARCHAR(255),
        publisher_name VARCHAR(255),
        first_name VARCHAR(100),
        middle_name VARCHAR(50),
        last_name VARCHAR(100)
    );
"""
print("Executing create table if not exists query:", create_table_if_not_exists_query)
session.execute(create_table_if_not_exists_query)
print("Executing select query (books JOIN publishers):")
books_publishers_query = """
    SELECT
        b.book_id,
        b.title,
        b.total_pages,
        b.rating,
        b.isbn,
        b.published_date,
        p.name AS publisher_name
    FROM
        books b
    LEFT JOIN
        publishers p ON b.publisher_id = p.publisher_id
"""
books_publishers_results = session.execute(books_publishers_query).fetchall()
for row in books_publishers_results:
    print(row)
print("Executing select query (books_publishers JOIN book_authors):")
books_publishers_authors_query = """
    SELECT
        bp.book_id,
        bp.title,
        bp.total_pages,
        bp.rating,
        bp.isbn,
        bp.published_date,
        bp.publisher_name,
        a.first_name,
        a.middle_name,
        a.last_name
    FROM
        (
            SELECT
                b.book_id,
                b.title,
                b.total_pages,
                b.rating,
                b.isbn,
                b.published_date,
                p.name AS publisher_name
            FROM
                books b
            LEFT JOIN
                publishers p ON b.publisher_id = p.publisher_id
        ) bp
    LEFT JOIN
        book_authors ba ON bp.book_id = ba.book_id
    LEFT JOIN
        authors a ON ba.author_id = a.author_id
"""
books_publishers_authors_results = session.execute(books_publishers_authors_query).fetchall()
for row in books_publishers_authors_results:
    print(row)
print("Llenado de la tabla unified_books")
insert_query = """
    INSERT INTO unified_books (
        book_id, title, total_pages, rating, isbn, published_date,
        publisher_name, first_name, middle_name, last_name
    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
raw_connection = engine.raw_connection()
raw_cursor = raw_connection.cursor()
for row in books_publishers_authors_results:
    params = (
        str(row[0]) if row[0] is not None else "0",
        str(row[1]) if row[1] is not None else "NULL",
        str(row[2]) if row[2] is not None else "0",
        str(row[3]) if row[3] is not None else "0",
        str(row[4]) if row[4] is not None else "NULL",
        str(row[5]) if row[5] is not None else "NULL",
        str(row[6]) if row[6] is not None else "NULL",
        str(row[7]) if row[7] is not None else "NULL",
        str(row[8]) if row[8] is not None else "NULL",
        str(row[9]) if row[9] is not None else "NULL",
    )
    raw_cursor.execute(insert_query, params)
    print("Inserted row:", params)
raw_connection.commit()
print("Tabla unificada creada")
raw_cursor.close()
raw_connection.close()
#Fin del paso 2.

Executing create table if not exists query: 
    CREATE TABLE IF NOT EXISTS unified_books (
        book_id VARCHAR(255),
        title VARCHAR(255),
        total_pages VARCHAR(255),
        rating VARCHAR(255),
        isbn VARCHAR(13),
        published_date VARCHAR(255),
        publisher_name VARCHAR(255),
        first_name VARCHAR(100),
        middle_name VARCHAR(50),
        last_name VARCHAR(100)
    );

Executing select query (books JOIN publishers):
(1, 'Lean Software Development: An Agile Toolkit', 240, Decimal('4.17'), '9780320000000', datetime.date(2003, 5, 18), 'Addison Wesley')
(2, 'Facing the Intelligence Explosion', 91, Decimal('3.87'), None, datetime.date(2013, 2, 1), 'Alfred A Knopf')
(3, 'Scala in Action', 419, Decimal('3.74'), '9781940000000', datetime.date(2013, 4, 10), 'O Reilly Media')
(4, 'Patterns of Software: Tales from the Software Community', 256, Decimal('3.84'), '9780200000000', datetime.date(1996, 8, 15), 'O Reilly Media')
(5, 'Anatomy Of LISP', 446, D

In [3]:
import os
import pandas as pd
import psycopg2
from dotenv import load_dotenv
from sqlalchemy import create_engine
from pathlib import Path
#Paso 3, realizamos un query con sintaxis sqlalchemy para cargar las tablas en dataframes.
current_dir = Path(os.getcwd())
dotenv_path = current_dir / 'user.env'
load_dotenv(dotenv_path=dotenv_path)
db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
DATABASE_URL = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(DATABASE_URL)
with engine.connect() as connection:
    publishers_results = connection.execute("SELECT * FROM publishers").fetchall()
    authors_results = connection.execute("SELECT * FROM authors").fetchall()
    books_results = connection.execute("SELECT * FROM books").fetchall()
    book_authors_results = connection.execute("SELECT * FROM book_authors").fetchall()
    unified_books_results = connection.execute("SELECT * FROM unified_books").fetchall()
df_publishers = pd.DataFrame(publishers_results, columns=["publisher_id", "name"])
df_authors = pd.DataFrame(authors_results, columns=["author_id", "first_name", "middle_name", "last_name"])
df_books = pd.DataFrame(books_results, columns=["book_id", "title", "total_pages", "rating", "isbn", "published_date", "publisher_id"])
df_book_authors = pd.DataFrame(book_authors_results, columns=["book_id", "author_id"])
df_unified_books = pd.DataFrame(unified_books_results, columns=["book_id", "title", "total_pages", "rating", "isbn", "published_date", "publisher_name", "first_name", "middle_name", "last_name"])
print(df_publishers.head(2), '\n')
print("-" * 50)
print(df_authors.head(2), '\n')
print("-" * 50)
print(df_books.head(2), '\n')
print("-" * 50)
print(df_book_authors.head(2), '\n')
print("-" * 50)
print(df_unified_books.head(2), '\n')
print("-" * 50)
#Fin del paso 3.

   publisher_id            name
0             1  O Reilly Media
1             2    A Book Apart 

--------------------------------------------------
   author_id first_name middle_name last_name
0          1    Merritt        None      Eric
1          2      Linda        None       Mui 

--------------------------------------------------
   book_id                                        title  total_pages rating  \
0        1  Lean Software Development: An Agile Toolkit          240   4.17   
1        2            Facing the Intelligence Explosion           91   3.87   

            isbn published_date  publisher_id  
0  9780320000000     2003-05-18             5  
1           None     2013-02-01             7   

--------------------------------------------------
   book_id  author_id
0        1          1
1        2          8 

--------------------------------------------------
   book_id                                        title  total_pages rating  \
0        1  Lean Software D

In [4]:
%reset -f